In [1]:
'''
This file plots CMIP5 RCP - HIST over South Australia
/Users/earl/Desktop/Yang
and places the outputs in
/Users/earl/Dropbox/CMIP5/figures

Earl Duran 
created: 19-Mar-18
e.duran@unsw.edu.au
'''

import cosima_cookbook as cc
import os
import xarray as xr
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib
import sys
import pickle
import itertools
def find_nearest_index(array, value):
    return int((np.abs(array - value)).argmin())
from scipy import interpolate
from scipy import stats
from dask.distributed import Client
import pandas as pd
from glob import glob as gb
import gsw

import cartopy.crs as ccrs
import cartopy.feature as cft
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
def arange(start,end,stride=1):
    return np.arange(start,end+0.00000001,stride)

import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata
print('Available exptdata keys: ', [k for k in exptdata.exptdict.keys()])

session = cc.database.create_session()

Available exptdata keys:  ['1deg', '025deg', '01deg']


In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:35079 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 33.67 GB


In [4]:
# input_path = '/g/data/e14/erd561/NOAA_OI_SST_V2/'

output_path = '/g/data/e14/erd561/Australia_3/'

In [5]:
%%time
lonW = 145-360
lonE = 175-360
latL = -34
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

temp_yflux_adv_ym34_1_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp_yflux_adv').\
interp(yu_ocean=latL).\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=slice(0,2100))
print(temp_yflux_adv_ym34_1_mod_month)

temp_yflux_adv_ym34_1_mod_month.load()
print(temp_yflux_adv_ym34_1_mod_month)

xt_ocean = temp_yflux_adv_ym34_1_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
temp_yflux_adv_ym34_1_mod_month = temp_yflux_adv_ym34_1_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
st_ocean = temp_yflux_adv_ym34_1_mod_month.st_ocean
st_ocean_corrected = -st_ocean
temp_yflux_adv_ym34_1_mod_month = temp_yflux_adv_ym34_1_mod_month.assign_coords(st_ocean=st_ocean_corrected)
print(temp_yflux_adv_ym34_1_mod_month)

temp_yflux_adv_ym34_1_mod = temp_yflux_adv_ym34_1_mod_month.groupby('time.year').mean('time')
print(temp_yflux_adv_ym34_1_mod)

temp_yflux_adv_ym34_1_mod = temp_yflux_adv_ym34_1_mod.sel(year=slice(2004+60*4,2017+60*4))
print(temp_yflux_adv_ym34_1_mod)

1deg_jra55v13_iaf_spinup1_B1


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'temp_yflux_adv' (time: 300, st_ocean: 35, xt_ocean: 30)>
dask.array<getitem, shape=(300, 35, 30), dtype=float32, chunksize=(1, 25, 30), chunktype=numpy.ndarray>
Coordinates:
  * st_ocean  (st_ocean) float64 1.152 3.649 6.565 ... 1.854e+03 2.046e+03
  * xt_ocean  (xt_ocean) float64 -214.5 -213.5 -212.5 ... -187.5 -186.5 -185.5
  * time      (time) object 1958-07-02 12:00:00 ... 2257-07-02 12:00:00
    yu_ocean  int64 -34
Attributes:
    long_name:      cp*rho*dzt*dxt*v*temp
    units:          Watts
    valid_range:    [-1.e+18  1.e+18]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_c
<xarray.DataArray 'temp_yflux_adv' (time: 300, st_ocean: 35, xt_ocean: 30)>
array([[[            nan,             nan,             nan, ...,
          7.35225219e+10,             nan, -6.83562729e+11],
        [            nan,             nan,             nan, ...,
          1.14277867e+11,             nan, -7.1456

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [6]:
year = temp_yflux_adv_ym34_1_mod.year
depth_1_mod = temp_yflux_adv_ym34_1_mod.st_ocean
lon_1_mod = temp_yflux_adv_ym34_1_mod.xt_ocean

temp_yflux_adv_ym34_1_trans_mod = temp_yflux_adv_ym34_1_mod
print(np.shape(temp_yflux_adv_ym34_1_trans_mod))
temp_yflux_adv_ym34_1_slope_mod = xr.DataArray(
    np.zeros([np.shape(depth_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'depth_1', 'lon_1'), coords=[depth_1_mod, lon_1_mod], name='temp_yflux_adv_ym34_1')
temp_yflux_adv_ym34_1_p_value_mod = xr.DataArray(
    np.zeros([np.shape(depth_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'depth_1', 'lon_1'), coords=[depth_1_mod, lon_1_mod], name='temp_yflux_adv_ym34_1')
temp_yflux_adv_ym34_1_std_err_mod = xr.DataArray(
    np.zeros([np.shape(depth_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'depth_1', 'lon_1'), coords=[depth_1_mod, lon_1_mod], name='temp_yflux_adv_ym34_1')
for iid, i in enumerate(depth_1_mod):
    for jid, j in enumerate(lon_1_mod):
        temp_yflux_adv_ym34_1_slope_mod[iid, jid], _, _, temp_yflux_adv_ym34_1_p_value_mod[iid, jid], temp_yflux_adv_ym34_1_std_err_mod[iid, jid] = \
        stats.linregress(year, temp_yflux_adv_ym34_1_trans_mod[:, iid, jid])
    print('depth_1_mod ' + str(np.array(i)))
print(temp_yflux_adv_ym34_1_slope_mod)

(14, 35, 30)
depth_1_mod -1.15174989182586
depth_1_mod -3.6486741197120196
depth_1_mod -6.564918471676153
depth_1_mod -9.970869194872916
depth_1_mod -13.948708373749158
depth_1_mod -18.594379460593196
depth_1_mod -24.019873617545198
depth_1_mod -30.355885242559452
depth_1_mod -37.754889932903495
depth_1_mod -46.39470194845016
depth_1_mod -56.48256969937929
depth_1_mod -68.25986482598941
depth_1_mod -82.00740970018322
depth_1_mod -98.05146439730592
depth_1_mod -116.77034928747688
depth_1_mod -138.60160118643546
depth_1_mod -164.04943216631892
depth_1_mod -193.69205664349184
depth_1_mod -228.1881444592732
depth_1_mod -268.2812141854562
depth_1_mod -314.800181990729
depth_1_mod -368.65354740782504
depth_1_mod -430.8139438640983
depth_1_mod -502.28929308570173
depth_1_mod -584.0771003035616
depth_1_mod -677.1002526914099
depth_1_mod -782.1267221440287
depth_1_mod -899.6817765561261
depth_1_mod -1029.9681126743835
depth_1_mod -1172.813371535859
depth_1_mod -1327.6620295174746
depth_1_mod -1

In [7]:
%%time
latL = -34
ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

temp_yflux_adv_ym34_025_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp_yflux_adv').\
interp(yu_ocean=latL).\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=slice(0,2100))
print(temp_yflux_adv_ym34_025_mod_month)

temp_yflux_adv_ym34_025_mod_month.load()
print(temp_yflux_adv_ym34_025_mod_month)

xt_ocean = temp_yflux_adv_ym34_025_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
temp_yflux_adv_ym34_025_mod_month = temp_yflux_adv_ym34_025_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
st_ocean = temp_yflux_adv_ym34_025_mod_month.st_ocean
st_ocean_corrected = -st_ocean
temp_yflux_adv_ym34_025_mod_month = temp_yflux_adv_ym34_025_mod_month.assign_coords(st_ocean=st_ocean_corrected)
print(temp_yflux_adv_ym34_025_mod_month)

temp_yflux_adv_ym34_025_mod = temp_yflux_adv_ym34_025_mod_month.groupby('time.year').mean('time')
print(temp_yflux_adv_ym34_025_mod)

temp_yflux_adv_ym34_025_mod = temp_yflux_adv_ym34_025_mod.sel(year=slice(2004+60*4,2017+60*4))
print(temp_yflux_adv_ym34_025_mod)

025deg_jra55v13_iaf_gmredi6


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'temp_yflux_adv' (time: 300, st_ocean: 35, xt_ocean: 120)>
dask.array<getitem, shape=(300, 35, 120), dtype=float32, chunksize=(1, 10, 92), chunktype=numpy.ndarray>
Coordinates:
  * st_ocean  (st_ocean) float64 1.152 3.649 6.565 ... 1.854e+03 2.046e+03
  * xt_ocean  (xt_ocean) float64 -214.9 -214.6 -214.4 ... -185.6 -185.4 -185.1
  * time      (time) object 1958-06-30 12:00:00 ... 2257-06-30 12:00:00
    yu_ocean  int64 -34
Attributes:
    long_name:      cp*rho*dzt*dxt*v*temp
    units:          Watts
    valid_range:    [-1.e+18  1.e+18]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_c
<xarray.DataArray 'temp_yflux_adv' (time: 300, st_ocean: 35, xt_ocean: 120)>
array([[[            nan,             nan,             nan, ...,
         -3.45073300e+11, -2.91015884e+11, -2.01776706e+11],
        [            nan,             nan,             nan, ...,
         -3.90513299e+11, -3.27450256e+11, -2.2

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [8]:
year = temp_yflux_adv_ym34_025_mod.year
depth_025_mod = temp_yflux_adv_ym34_025_mod.st_ocean
lon_025_mod = temp_yflux_adv_ym34_025_mod.xt_ocean

temp_yflux_adv_ym34_025_trans_mod = temp_yflux_adv_ym34_025_mod
print(np.shape(temp_yflux_adv_ym34_025_trans_mod))
temp_yflux_adv_ym34_025_slope_mod = xr.DataArray(
    np.zeros([np.shape(depth_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'depth_025', 'lon_025'), coords=[depth_025_mod, lon_025_mod], name='temp_yflux_adv_ym34_025')
temp_yflux_adv_ym34_025_p_value_mod = xr.DataArray(
    np.zeros([np.shape(depth_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'depth_025', 'lon_025'), coords=[depth_025_mod, lon_025_mod], name='temp_yflux_adv_ym34_025')
temp_yflux_adv_ym34_025_std_err_mod = xr.DataArray(
    np.zeros([np.shape(depth_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'depth_025', 'lon_025'), coords=[depth_025_mod, lon_025_mod], name='temp_yflux_adv_ym34_025')
for iid, i in enumerate(depth_025_mod):
    for jid, j in enumerate(lon_025_mod):
        temp_yflux_adv_ym34_025_slope_mod[iid, jid], _, _, temp_yflux_adv_ym34_025_p_value_mod[iid, jid], temp_yflux_adv_ym34_025_std_err_mod[iid, jid] = \
        stats.linregress(year, temp_yflux_adv_ym34_025_trans_mod[:, iid, jid])
    print('depth_025_mod ' + str(np.array(i)))
print(temp_yflux_adv_ym34_025_slope_mod)

(14, 35, 120)
depth_025_mod -1.15174989182586
depth_025_mod -3.6486741197120196
depth_025_mod -6.564918471676153
depth_025_mod -9.970869194872916
depth_025_mod -13.948708373749158
depth_025_mod -18.594379460593196
depth_025_mod -24.019873617545198
depth_025_mod -30.355885242559452
depth_025_mod -37.754889932903495
depth_025_mod -46.39470194845016
depth_025_mod -56.48256969937929
depth_025_mod -68.25986482598941
depth_025_mod -82.00740970018322
depth_025_mod -98.05146439730592
depth_025_mod -116.77034928747688
depth_025_mod -138.60160118643546
depth_025_mod -164.04943216631892
depth_025_mod -193.69205664349184
depth_025_mod -228.1881444592732
depth_025_mod -268.2812141854562
depth_025_mod -314.800181990729
depth_025_mod -368.65354740782504
depth_025_mod -430.8139438640983
depth_025_mod -502.28929308570173
depth_025_mod -584.0771003035616
depth_025_mod -677.1002526914099
depth_025_mod -782.1267221440287
depth_025_mod -899.6817765561261
depth_025_mod -1029.9681126743835
depth_025_mod -117

In [9]:
%%time
latL = -34
ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

temp_yflux_adv_ym34_01_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp_yflux_adv').\
sel(yu_ocean=latL, method='nearest').\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=slice(0,2100))
print(temp_yflux_adv_ym34_01_mod_month)

temp_yflux_adv_ym34_01_mod_month.load()
print(temp_yflux_adv_ym34_01_mod_month)

xt_ocean = temp_yflux_adv_ym34_01_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
temp_yflux_adv_ym34_01_mod_month = temp_yflux_adv_ym34_01_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
st_ocean = temp_yflux_adv_ym34_01_mod_month.st_ocean
st_ocean_corrected = -st_ocean
temp_yflux_adv_ym34_01_mod_month = temp_yflux_adv_ym34_01_mod_month.assign_coords(st_ocean=st_ocean_corrected)
print(temp_yflux_adv_ym34_01_mod_month)

temp_yflux_adv_ym34_01_mod = temp_yflux_adv_ym34_01_mod_month.groupby('time.year').mean('time')
print(temp_yflux_adv_ym34_01_mod)

temp_yflux_adv_ym34_01_mod = temp_yflux_adv_ym34_01_mod.sel(year=slice(2004,2017))
print(temp_yflux_adv_ym34_01_mod)

01deg_jra55v13_iaf


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'temp_yflux_adv' (time: 396, st_ocean: 55, xt_ocean: 300)>
dask.array<getitem, shape=(396, 55, 300), dtype=float32, chunksize=(1, 7, 150), chunktype=numpy.ndarray>
Coordinates:
  * st_ocean  (st_ocean) float64 0.5413 1.681 2.94 ... 1.858e+03 2.007e+03
  * xt_ocean  (xt_ocean) float64 -214.9 -214.8 -214.7 ... -185.2 -185.1 -185.0
    yu_ocean  float64 -34.01
  * time      (time) object 1985-01-14 12:00:00 ... 2017-12-14 12:00:00
Attributes:
    long_name:      cp*rho*dzt*dxt*v*temp
    units:          Watts
    valid_range:    [-1.e+18  1.e+18]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_c
<xarray.DataArray 'temp_yflux_adv' (time: 396, st_ocean: 55, xt_ocean: 300)>
array([[[           nan,            nan,            nan, ...,
          4.4503593e+10,  3.4555589e+10,  2.2734025e+10],
        [           nan,            nan,            nan, ...,
          4.6906270e+10,  3.5840778e+10,  2.2698375

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [10]:
year = temp_yflux_adv_ym34_01_mod.year
depth_01_mod = temp_yflux_adv_ym34_01_mod.st_ocean
lon_01_mod = temp_yflux_adv_ym34_01_mod.xt_ocean

temp_yflux_adv_ym34_01_trans_mod = temp_yflux_adv_ym34_01_mod
print(np.shape(temp_yflux_adv_ym34_01_trans_mod))
temp_yflux_adv_ym34_01_slope_mod = xr.DataArray(
    np.zeros([np.shape(depth_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'depth_01', 'lon_01'), coords=[depth_01_mod, lon_01_mod], name='temp_yflux_adv_ym34_01')
temp_yflux_adv_ym34_01_p_value_mod = xr.DataArray(
    np.zeros([np.shape(depth_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'depth_01', 'lon_01'), coords=[depth_01_mod, lon_01_mod], name='temp_yflux_adv_ym34_01')
temp_yflux_adv_ym34_01_std_err_mod = xr.DataArray(
    np.zeros([np.shape(depth_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'depth_01', 'lon_01'), coords=[depth_01_mod, lon_01_mod], name='temp_yflux_adv_ym34_01')
for iid, i in enumerate(depth_01_mod):
    for jid, j in enumerate(lon_01_mod):
        temp_yflux_adv_ym34_01_slope_mod[iid, jid], _, _, temp_yflux_adv_ym34_01_p_value_mod[iid, jid], temp_yflux_adv_ym34_01_std_err_mod[iid, jid] = \
        stats.linregress(year, temp_yflux_adv_ym34_01_trans_mod[:, iid, jid])
    print('depth_01_mod ' + str(np.array(i)))
print(temp_yflux_adv_ym34_01_slope_mod)

(14, 55, 300)
depth_01_mod -0.5412807653916101
depth_01_mod -1.680734679831433
depth_01_mod -2.939952648914465
depth_01_mod -4.331521485149508
depth_01_mod -5.8693504240540255
depth_01_mod -7.5688099200502155
depth_01_mod -9.446884959648713
depth_01_mod -11.522344392803275
depth_01_mod -13.815927932333222
depth_01_mod -16.350552632935347
depth_01_mod -19.151540835805992
depth_01_mod -22.24687175218868
depth_01_mod -25.667459058670023
depth_01_mod -29.447457087455206
depth_01_mod -33.62459841385048
depth_01_mod -38.24056586677453
depth_01_mod -43.341402211041604
depth_01_mod -48.97796096317406
depth_01_mod -55.20640199365842
depth_01_mod -62.088735720175855
depth_01_mod -69.69341978534455
depth_01_mod -78.09601210263148
depth_01_mod -87.37988400213499
depth_01_mod -97.63699685168024
depth_01_mod -108.96874488209758
depth_01_mod -121.48686590184016
depth_01_mod -135.31441999260392
depth_01_mod -150.5868339523135
depth_01_mod -167.45300594349283
depth_01_mod -186.07646019149857
depth_01_m

In [11]:
temp_yflux_adv_ym34_1_mod_xr = xr.DataArray(temp_yflux_adv_ym34_1_mod.transpose('st_ocean', 'xt_ocean', 'year'), name='temp_yflux_adv_ym34_1_mod',
                        coords=[depth_1_mod, lon_1_mod, year], 
                        dims=['st_ocean', 'xt_ocean', 'year'])
print(temp_yflux_adv_ym34_1_mod_xr)
temp_yflux_adv_ym34_1_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym34_1_mod.nc')

temp_yflux_adv_ym34_1_slope_mod_xr = xr.DataArray(temp_yflux_adv_ym34_1_slope_mod, name='temp_yflux_adv_ym34_1_slope_mod',
                        coords=[depth_1_mod, lon_1_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_yflux_adv_ym34_1_slope_mod_xr)
temp_yflux_adv_ym34_1_slope_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym34_1_slope_mod.nc')

temp_yflux_adv_ym34_1_p_value_mod_xr = xr.DataArray(temp_yflux_adv_ym34_1_p_value_mod, name='temp_yflux_adv_ym34_1_p_value_mod',
                        coords=[depth_1_mod, lon_1_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_yflux_adv_ym34_1_p_value_mod_xr)
temp_yflux_adv_ym34_1_p_value_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym34_1_p_value_mod.nc')

temp_yflux_adv_ym34_1_std_err_mod_xr = xr.DataArray(temp_yflux_adv_ym34_1_std_err_mod, name='temp_yflux_adv_ym34_1_std_err_mod',
                        coords=[depth_1_mod, lon_1_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_yflux_adv_ym34_1_std_err_mod_xr)
temp_yflux_adv_ym34_1_std_err_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym34_1_std_err_mod.nc')



temp_yflux_adv_ym34_025_mod_xr = xr.DataArray(temp_yflux_adv_ym34_025_mod.transpose('st_ocean', 'xt_ocean', 'year'), name='temp_yflux_adv_ym34_025_mod',
                        coords=[depth_025_mod, lon_025_mod, year], 
                        dims=['st_ocean', 'xt_ocean', 'year'])
print(temp_yflux_adv_ym34_025_mod_xr)
temp_yflux_adv_ym34_025_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym34_025_mod.nc')

temp_yflux_adv_ym34_025_slope_mod_xr = xr.DataArray(temp_yflux_adv_ym34_025_slope_mod, name='temp_yflux_adv_ym34_025_slope_mod',
                        coords=[depth_025_mod, lon_025_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_yflux_adv_ym34_025_slope_mod_xr)
temp_yflux_adv_ym34_025_slope_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym34_025_slope_mod.nc')

temp_yflux_adv_ym34_025_p_value_mod_xr = xr.DataArray(temp_yflux_adv_ym34_025_p_value_mod, name='temp_yflux_adv_ym34_025_p_value_mod',
                        coords=[depth_025_mod, lon_025_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_yflux_adv_ym34_025_p_value_mod_xr)
temp_yflux_adv_ym34_025_p_value_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym34_025_p_value_mod.nc')

temp_yflux_adv_ym34_025_std_err_mod_xr = xr.DataArray(temp_yflux_adv_ym34_025_std_err_mod, name='temp_yflux_adv_ym34_025_std_err_mod',
                        coords=[depth_025_mod, lon_025_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_yflux_adv_ym34_025_std_err_mod_xr)
temp_yflux_adv_ym34_025_std_err_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym34_025_std_err_mod.nc')



temp_yflux_adv_ym34_01_mod_xr = xr.DataArray(temp_yflux_adv_ym34_01_mod.transpose('st_ocean', 'xt_ocean', 'year'), name='temp_yflux_adv_ym34_01_mod',
                        coords=[depth_01_mod, lon_01_mod, year], 
                        dims=['st_ocean', 'xt_ocean', 'year'])
print(temp_yflux_adv_ym34_01_mod_xr)
temp_yflux_adv_ym34_01_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym34_01_mod.nc')

temp_yflux_adv_ym34_01_slope_mod_xr = xr.DataArray(temp_yflux_adv_ym34_01_slope_mod, name='temp_yflux_adv_ym34_01_slope_mod',
                        coords=[depth_01_mod, lon_01_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_yflux_adv_ym34_01_slope_mod_xr)
temp_yflux_adv_ym34_01_slope_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym34_01_slope_mod.nc')

temp_yflux_adv_ym34_01_p_value_mod_xr = xr.DataArray(temp_yflux_adv_ym34_01_p_value_mod, name='temp_yflux_adv_ym34_01_p_value_mod',
                        coords=[depth_01_mod, lon_01_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_yflux_adv_ym34_01_p_value_mod_xr)
temp_yflux_adv_ym34_01_p_value_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym34_01_p_value_mod.nc')

temp_yflux_adv_ym34_01_std_err_mod_xr = xr.DataArray(temp_yflux_adv_ym34_01_std_err_mod, name='temp_yflux_adv_ym34_01_std_err_mod',
                        coords=[depth_01_mod, lon_01_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_yflux_adv_ym34_01_std_err_mod_xr)
temp_yflux_adv_ym34_01_std_err_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym34_01_std_err_mod.nc')

<xarray.DataArray 'temp_yflux_adv_ym34_1_mod' (st_ocean: 35, xt_ocean: 30, year: 14)>
array([[[            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        ...,
        [ 3.98171508e+11,  2.36073452e+11,  3.98757790e+11, ...,
          3.00597814e+11,  1.33541277e+11,  2.06845482e+11],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [-4.03041994e+11, -6.98027078e+11, -5.02220844e+11, ...,
         -6.63549141e+11, -8.18464368e+11, -6.99449716e+11]],

       [[            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [        

<xarray.DataArray 'temp_yflux_adv_ym34_025_p_value_mod' (st_ocean: 35, xt_ocean: 120)>
array([[       nan,        nan,        nan, ..., 0.28259455, 0.32190917,
        0.33555167],
       [       nan,        nan,        nan, ..., 0.26888489, 0.3041726 ,
        0.31361673],
       [       nan,        nan,        nan, ..., 0.25590931, 0.28759011,
        0.29526277],
       ...,
       [       nan,        nan,        nan, ..., 0.45639563, 0.96609012,
        0.31439886],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan]])
Coordinates:
  * st_ocean  (st_ocean) float64 -1.152 -3.649 -6.565 ... -1.854e+03 -2.046e+03
  * xt_ocean  (xt_ocean) float64 145.1 145.4 145.6 145.9 ... 174.4 174.6 174.9
<xarray.DataArray 'temp_yflux_adv_ym34_025_std_err_mod' (st_ocean: 35, xt_ocean: 120)>
array([[           nan,            nan,            nan, ...,
        7.265044

<xarray.DataArray 'temp_yflux_adv_ym34_01_std_err_mod' (st_ocean: 55, xt_ocean: 300)>
array([[           nan,            nan,            nan, ...,
        1.67289150e+09, 1.52797034e+09, 1.51063127e+09],
       [           nan,            nan,            nan, ...,
        1.81739958e+09, 1.65925056e+09, 1.64178117e+09],
       [           nan,            nan,            nan, ...,
        1.98763464e+09, 1.81406087e+09, 1.79547929e+09],
       ...,
       [           nan,            nan,            nan, ...,
                   nan,            nan,            nan],
       [           nan,            nan,            nan, ...,
                   nan,            nan,            nan],
       [           nan,            nan,            nan, ...,
                   nan,            nan,            nan]])
Coordinates:
  * st_ocean  (st_ocean) float64 -0.5413 -1.681 -2.94 ... -1.858e+03 -2.007e+03
  * xt_ocean  (xt_ocean) float64 145.1 145.2 145.3 145.4 ... 174.8 174.9 175.0


In [14]:
%%time
latL = -42
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

temp_yflux_adv_ym42_1_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp_yflux_adv').\
interp(yu_ocean=latL).\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=slice(0,2100))
print(temp_yflux_adv_ym42_1_mod_month)

temp_yflux_adv_ym42_1_mod_month.load()
print(temp_yflux_adv_ym42_1_mod_month)

xt_ocean = temp_yflux_adv_ym42_1_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
temp_yflux_adv_ym42_1_mod_month = temp_yflux_adv_ym42_1_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
st_ocean = temp_yflux_adv_ym42_1_mod_month.st_ocean
st_ocean_corrected = -st_ocean
temp_yflux_adv_ym42_1_mod_month = temp_yflux_adv_ym42_1_mod_month.assign_coords(st_ocean=st_ocean_corrected)
print(temp_yflux_adv_ym42_1_mod_month)

temp_yflux_adv_ym42_1_mod = temp_yflux_adv_ym42_1_mod_month.groupby('time.year').mean('time')
print(temp_yflux_adv_ym42_1_mod)

temp_yflux_adv_ym42_1_mod = temp_yflux_adv_ym42_1_mod.sel(year=slice(2004+60*4,2017+60*4))
print(temp_yflux_adv_ym42_1_mod)


ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

temp_yflux_adv_ym42_025_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp_yflux_adv').\
interp(yu_ocean=latL).\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=slice(0,2100))
print(temp_yflux_adv_ym42_025_mod_month)

temp_yflux_adv_ym42_025_mod_month.load()
print(temp_yflux_adv_ym42_025_mod_month)

xt_ocean = temp_yflux_adv_ym42_025_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
temp_yflux_adv_ym42_025_mod_month = temp_yflux_adv_ym42_025_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
st_ocean = temp_yflux_adv_ym42_025_mod_month.st_ocean
st_ocean_corrected = -st_ocean
temp_yflux_adv_ym42_025_mod_month = temp_yflux_adv_ym42_025_mod_month.assign_coords(st_ocean=st_ocean_corrected)
print(temp_yflux_adv_ym42_025_mod_month)

temp_yflux_adv_ym42_025_mod = temp_yflux_adv_ym42_025_mod_month.groupby('time.year').mean('time')
print(temp_yflux_adv_ym42_025_mod)

temp_yflux_adv_ym42_025_mod = temp_yflux_adv_ym42_025_mod.sel(year=slice(2004+60*4,2017+60*4))
print(temp_yflux_adv_ym42_025_mod)


ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

temp_yflux_adv_ym42_01_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp_yflux_adv').\
sel(yu_ocean=latL, method='nearest').\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=slice(0,2100))
print(temp_yflux_adv_ym42_01_mod_month)

temp_yflux_adv_ym42_01_mod_month.load()
print(temp_yflux_adv_ym42_01_mod_month)

xt_ocean = temp_yflux_adv_ym42_01_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
temp_yflux_adv_ym42_01_mod_month = temp_yflux_adv_ym42_01_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
st_ocean = temp_yflux_adv_ym42_01_mod_month.st_ocean
st_ocean_corrected = -st_ocean
temp_yflux_adv_ym42_01_mod_month = temp_yflux_adv_ym42_01_mod_month.assign_coords(st_ocean=st_ocean_corrected)
print(temp_yflux_adv_ym42_01_mod_month)

temp_yflux_adv_ym42_01_mod = temp_yflux_adv_ym42_01_mod_month.groupby('time.year').mean('time')
print(temp_yflux_adv_ym42_01_mod)

temp_yflux_adv_ym42_01_mod = temp_yflux_adv_ym42_01_mod.sel(year=slice(2004,2017))
print(temp_yflux_adv_ym42_01_mod)


year = temp_yflux_adv_ym42_1_mod.year
depth_1_mod = temp_yflux_adv_ym42_1_mod.st_ocean
lon_1_mod = temp_yflux_adv_ym42_1_mod.xt_ocean
temp_yflux_adv_ym42_1_trans_mod = temp_yflux_adv_ym42_1_mod
print(np.shape(temp_yflux_adv_ym42_1_trans_mod))
temp_yflux_adv_ym42_1_slope_mod = xr.DataArray(
    np.zeros([np.shape(depth_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'depth_1', 'lon_1'), coords=[depth_1_mod, lon_1_mod], name='temp_yflux_adv_ym42_1')
temp_yflux_adv_ym42_1_p_value_mod = xr.DataArray(
    np.zeros([np.shape(depth_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'depth_1', 'lon_1'), coords=[depth_1_mod, lon_1_mod], name='temp_yflux_adv_ym42_1')
temp_yflux_adv_ym42_1_std_err_mod = xr.DataArray(
    np.zeros([np.shape(depth_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'depth_1', 'lon_1'), coords=[depth_1_mod, lon_1_mod], name='temp_yflux_adv_ym42_1')
for iid, i in enumerate(depth_1_mod):
    for jid, j in enumerate(lon_1_mod):
        temp_yflux_adv_ym42_1_slope_mod[iid, jid], _, _, temp_yflux_adv_ym42_1_p_value_mod[iid, jid], temp_yflux_adv_ym42_1_std_err_mod[iid, jid] = \
        stats.linregress(year, temp_yflux_adv_ym42_1_trans_mod[:, iid, jid])
    print('depth_1_mod ' + str(np.array(i)))
print(temp_yflux_adv_ym42_1_slope_mod)
year = temp_yflux_adv_ym42_025_mod.year
depth_025_mod = temp_yflux_adv_ym42_025_mod.st_ocean
lon_025_mod = temp_yflux_adv_ym42_025_mod.xt_ocean
temp_yflux_adv_ym42_025_trans_mod = temp_yflux_adv_ym42_025_mod
print(np.shape(temp_yflux_adv_ym42_025_trans_mod))
temp_yflux_adv_ym42_025_slope_mod = xr.DataArray(
    np.zeros([np.shape(depth_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'depth_025', 'lon_025'), coords=[depth_025_mod, lon_025_mod], name='temp_yflux_adv_ym42_025')
temp_yflux_adv_ym42_025_p_value_mod = xr.DataArray(
    np.zeros([np.shape(depth_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'depth_025', 'lon_025'), coords=[depth_025_mod, lon_025_mod], name='temp_yflux_adv_ym42_025')
temp_yflux_adv_ym42_025_std_err_mod = xr.DataArray(
    np.zeros([np.shape(depth_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'depth_025', 'lon_025'), coords=[depth_025_mod, lon_025_mod], name='temp_yflux_adv_ym42_025')
for iid, i in enumerate(depth_025_mod):
    for jid, j in enumerate(lon_025_mod):
        temp_yflux_adv_ym42_025_slope_mod[iid, jid], _, _, temp_yflux_adv_ym42_025_p_value_mod[iid, jid], temp_yflux_adv_ym42_025_std_err_mod[iid, jid] = \
        stats.linregress(year, temp_yflux_adv_ym42_025_trans_mod[:, iid, jid])
    print('depth_025_mod ' + str(np.array(i)))
print(temp_yflux_adv_ym42_025_slope_mod)
year = temp_yflux_adv_ym42_01_mod.year
depth_01_mod = temp_yflux_adv_ym42_01_mod.st_ocean
lon_01_mod = temp_yflux_adv_ym42_01_mod.xt_ocean
temp_yflux_adv_ym42_01_trans_mod = temp_yflux_adv_ym42_01_mod
print(np.shape(temp_yflux_adv_ym42_01_trans_mod))
temp_yflux_adv_ym42_01_slope_mod = xr.DataArray(
    np.zeros([np.shape(depth_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'depth_01', 'lon_01'), coords=[depth_01_mod, lon_01_mod], name='temp_yflux_adv_ym42_01')
temp_yflux_adv_ym42_01_p_value_mod = xr.DataArray(
    np.zeros([np.shape(depth_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'depth_01', 'lon_01'), coords=[depth_01_mod, lon_01_mod], name='temp_yflux_adv_ym42_01')
temp_yflux_adv_ym42_01_std_err_mod = xr.DataArray(
    np.zeros([np.shape(depth_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'depth_01', 'lon_01'), coords=[depth_01_mod, lon_01_mod], name='temp_yflux_adv_ym42_01')
for iid, i in enumerate(depth_01_mod):
    for jid, j in enumerate(lon_01_mod):
        temp_yflux_adv_ym42_01_slope_mod[iid, jid], _, _, temp_yflux_adv_ym42_01_p_value_mod[iid, jid], temp_yflux_adv_ym42_01_std_err_mod[iid, jid] = \
        stats.linregress(year, temp_yflux_adv_ym42_01_trans_mod[:, iid, jid])
    print('depth_01_mod ' + str(np.array(i)))
print(temp_yflux_adv_ym42_01_slope_mod)

1deg_jra55v13_iaf_spinup1_B1


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'temp_yflux_adv' (time: 300, st_ocean: 35, xt_ocean: 30)>
dask.array<getitem, shape=(300, 35, 30), dtype=float32, chunksize=(1, 25, 30), chunktype=numpy.ndarray>
Coordinates:
  * st_ocean  (st_ocean) float64 1.152 3.649 6.565 ... 1.854e+03 2.046e+03
  * xt_ocean  (xt_ocean) float64 -214.5 -213.5 -212.5 ... -187.5 -186.5 -185.5
  * time      (time) object 1958-07-02 12:00:00 ... 2257-07-02 12:00:00
    yu_ocean  int64 -42
Attributes:
    long_name:      cp*rho*dzt*dxt*v*temp
    units:          Watts
    valid_range:    [-1.e+18  1.e+18]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_c
<xarray.DataArray 'temp_yflux_adv' (time: 300, st_ocean: 35, xt_ocean: 30)>
array([[[ 8.33539238e+07,             nan,             nan, ...,
                     nan,             nan,  5.27883095e+09],
        [-5.83712213e+08,             nan,             nan, ...,
                     nan,             nan,  7.9178

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
befo

<xarray.DataArray 'temp_yflux_adv' (time: 300, st_ocean: 35, xt_ocean: 120)>
dask.array<getitem, shape=(300, 35, 120), dtype=float32, chunksize=(1, 10, 92), chunktype=numpy.ndarray>
Coordinates:
  * st_ocean  (st_ocean) float64 1.152 3.649 6.565 ... 1.854e+03 2.046e+03
  * xt_ocean  (xt_ocean) float64 -214.9 -214.6 -214.4 ... -185.6 -185.4 -185.1
  * time      (time) object 1958-06-30 12:00:00 ... 2257-06-30 12:00:00
    yu_ocean  int64 -42
Attributes:
    long_name:      cp*rho*dzt*dxt*v*temp
    units:          Watts
    valid_range:    [-1.e+18  1.e+18]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_c
<xarray.DataArray 'temp_yflux_adv' (time: 300, st_ocean: 35, xt_ocean: 120)>
array([[[ 9.25458950e+08,             nan,             nan, ...,
         -8.92894573e+09, -2.83828430e+10, -6.92135101e+10],
        [-1.27411326e+09,             nan,             nan, ...,
          2.92265209e+09, -9.79669471e+09, -5.5

...

<xarray.DataArray 'temp_yflux_adv' (year: 300, st_ocean: 35, xt_ocean: 120)>
array([[[ 9.25458950e+08,             nan,             nan, ...,
         -8.92894573e+09, -2.83828430e+10, -6.92135101e+10],
        [-1.27411326e+09,             nan,             nan, ...,
          2.92265209e+09, -9.79669471e+09, -5.54198069e+10],
        [-4.69674960e+09,             nan,             nan, ...,
          1.36579422e+10,  6.11778097e+09, -4.64109548e+10],
        ...,
        [            nan,             nan,             nan, ...,
                     nan,             nan,  5.92168202e+08],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan]],

       [[-5.72487886e+09,             nan,             nan, ...,
         -1.06230671e+11, -1.45492419e+11, -1.07840440e+11],
        [-9.74202266e+09, 

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'temp_yflux_adv' (time: 396, st_ocean: 55, xt_ocean: 300)>
dask.array<getitem, shape=(396, 55, 300), dtype=float32, chunksize=(1, 7, 150), chunktype=numpy.ndarray>
Coordinates:
  * st_ocean  (st_ocean) float64 0.5413 1.681 2.94 ... 1.858e+03 2.007e+03
  * xt_ocean  (xt_ocean) float64 -214.9 -214.8 -214.7 ... -185.2 -185.1 -185.0
    yu_ocean  float64 -42.03
  * time      (time) object 1985-01-14 12:00:00 ... 2017-12-14 12:00:00
Attributes:
    long_name:      cp*rho*dzt*dxt*v*temp
    units:          Watts
    valid_range:    [-1.e+18  1.e+18]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_c
<xarray.DataArray 'temp_yflux_adv' (time: 396, st_ocean: 55, xt_ocean: 300)>
array([[[ 1.79655700e+10,  1.07239506e+10,  0.00000000e+00, ...,
         -1.42394950e+11, -9.14791875e+10, -3.72497490e+10],
        [ 1.56296509e+10,  9.67729766e+09,  0.00000000e+00, ...,
         -1.44191816e+11, -8.75052564e+10,

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


depth_1_mod -3.6486741197120196
depth_1_mod -6.564918471676153
depth_1_mod -9.970869194872916
depth_1_mod -13.948708373749158
depth_1_mod -18.594379460593196
depth_1_mod -24.019873617545198
depth_1_mod -30.355885242559452
depth_1_mod -37.754889932903495
depth_1_mod -46.39470194845016
depth_1_mod -56.48256969937929
depth_1_mod -68.25986482598941
depth_1_mod -82.00740970018322
depth_1_mod -98.05146439730592
depth_1_mod -116.77034928747688
depth_1_mod -138.60160118643546
depth_1_mod -164.04943216631892
depth_1_mod -193.69205664349184
depth_1_mod -228.1881444592732
depth_1_mod -268.2812141854562
depth_1_mod -314.800181990729
depth_1_mod -368.65354740782504
depth_1_mod -430.8139438640983
depth_1_mod -502.28929308570173
depth_1_mod -584.0771003035616
depth_1_mod -677.1002526914099
depth_1_mod -782.1267221440287
depth_1_mod -899.6817765561261
depth_1_mod -1029.9681126743835
depth_1_mod -1172.813371535859
depth_1_mod -1327.6620295174746
depth_1_mod -1493.618454761941
depth_1_mod -1669.53339696

In [15]:
temp_yflux_adv_ym42_1_mod_xr = xr.DataArray(temp_yflux_adv_ym42_1_mod.transpose('st_ocean', 'xt_ocean', 'year'), name='temp_yflux_adv_ym42_1_mod',
                        coords=[depth_1_mod, lon_1_mod, year], 
                        dims=['st_ocean', 'xt_ocean', 'year'])
print(temp_yflux_adv_ym42_1_mod_xr)
temp_yflux_adv_ym42_1_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym42_1_mod.nc')

temp_yflux_adv_ym42_1_slope_mod_xr = xr.DataArray(temp_yflux_adv_ym42_1_slope_mod, name='temp_yflux_adv_ym42_1_slope_mod',
                        coords=[depth_1_mod, lon_1_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_yflux_adv_ym42_1_slope_mod_xr)
temp_yflux_adv_ym42_1_slope_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym42_1_slope_mod.nc')

temp_yflux_adv_ym42_1_p_value_mod_xr = xr.DataArray(temp_yflux_adv_ym42_1_p_value_mod, name='temp_yflux_adv_ym42_1_p_value_mod',
                        coords=[depth_1_mod, lon_1_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_yflux_adv_ym42_1_p_value_mod_xr)
temp_yflux_adv_ym42_1_p_value_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym42_1_p_value_mod.nc')

temp_yflux_adv_ym42_1_std_err_mod_xr = xr.DataArray(temp_yflux_adv_ym42_1_std_err_mod, name='temp_yflux_adv_ym42_1_std_err_mod',
                        coords=[depth_1_mod, lon_1_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_yflux_adv_ym42_1_std_err_mod_xr)
temp_yflux_adv_ym42_1_std_err_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym42_1_std_err_mod.nc')



temp_yflux_adv_ym42_025_mod_xr = xr.DataArray(temp_yflux_adv_ym42_025_mod.transpose('st_ocean', 'xt_ocean', 'year'), name='temp_yflux_adv_ym42_025_mod',
                        coords=[depth_025_mod, lon_025_mod, year], 
                        dims=['st_ocean', 'xt_ocean', 'year'])
print(temp_yflux_adv_ym42_025_mod_xr)
temp_yflux_adv_ym42_025_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym42_025_mod.nc')

temp_yflux_adv_ym42_025_slope_mod_xr = xr.DataArray(temp_yflux_adv_ym42_025_slope_mod, name='temp_yflux_adv_ym42_025_slope_mod',
                        coords=[depth_025_mod, lon_025_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_yflux_adv_ym42_025_slope_mod_xr)
temp_yflux_adv_ym42_025_slope_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym42_025_slope_mod.nc')

temp_yflux_adv_ym42_025_p_value_mod_xr = xr.DataArray(temp_yflux_adv_ym42_025_p_value_mod, name='temp_yflux_adv_ym42_025_p_value_mod',
                        coords=[depth_025_mod, lon_025_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_yflux_adv_ym42_025_p_value_mod_xr)
temp_yflux_adv_ym42_025_p_value_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym42_025_p_value_mod.nc')

temp_yflux_adv_ym42_025_std_err_mod_xr = xr.DataArray(temp_yflux_adv_ym42_025_std_err_mod, name='temp_yflux_adv_ym42_025_std_err_mod',
                        coords=[depth_025_mod, lon_025_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_yflux_adv_ym42_025_std_err_mod_xr)
temp_yflux_adv_ym42_025_std_err_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym42_025_std_err_mod.nc')



temp_yflux_adv_ym42_01_mod_xr = xr.DataArray(temp_yflux_adv_ym42_01_mod.transpose('st_ocean', 'xt_ocean', 'year'), name='temp_yflux_adv_ym42_01_mod',
                        coords=[depth_01_mod, lon_01_mod, year], 
                        dims=['st_ocean', 'xt_ocean', 'year'])
print(temp_yflux_adv_ym42_01_mod_xr)
temp_yflux_adv_ym42_01_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym42_01_mod.nc')

temp_yflux_adv_ym42_01_slope_mod_xr = xr.DataArray(temp_yflux_adv_ym42_01_slope_mod, name='temp_yflux_adv_ym42_01_slope_mod',
                        coords=[depth_01_mod, lon_01_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_yflux_adv_ym42_01_slope_mod_xr)
temp_yflux_adv_ym42_01_slope_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym42_01_slope_mod.nc')

temp_yflux_adv_ym42_01_p_value_mod_xr = xr.DataArray(temp_yflux_adv_ym42_01_p_value_mod, name='temp_yflux_adv_ym42_01_p_value_mod',
                        coords=[depth_01_mod, lon_01_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_yflux_adv_ym42_01_p_value_mod_xr)
temp_yflux_adv_ym42_01_p_value_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym42_01_p_value_mod.nc')

temp_yflux_adv_ym42_01_std_err_mod_xr = xr.DataArray(temp_yflux_adv_ym42_01_std_err_mod, name='temp_yflux_adv_ym42_01_std_err_mod',
                        coords=[depth_01_mod, lon_01_mod], 
                        dims=['st_ocean', 'xt_ocean'])
print(temp_yflux_adv_ym42_01_std_err_mod_xr)
temp_yflux_adv_ym42_01_std_err_mod_xr.to_netcdf(output_path + 'temp_yflux_adv_ym42_01_std_err_mod.nc')

<xarray.DataArray 'temp_yflux_adv_ym42_1_mod' (st_ocean: 35, xt_ocean: 30, year: 14)>
array([[[ 2.31201667e+09, -1.49706683e+10, -1.06852922e+10, ...,
         -7.25145354e+08, -1.15222569e+10, -1.07143820e+10],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        ...,
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [ 5.44120360e+09,  1.23893806e+09,  6.89100018e+09, ...,
          2.32304868e+09, -2.51940582e+09,  1.30179475e+09]],

       [[ 2.39299324e+09, -1.32721272e+10, -1.25085995e+10, ...,
         -1.64462123e+09, -1.23334865e+10, -1.00328401e+10],
        [        

<xarray.DataArray 'temp_yflux_adv_ym42_01_p_value_mod' (st_ocean: 55, xt_ocean: 300)>
array([[0.35917508, 0.51518855, 1.        , ..., 0.84762327, 0.63610763,
        0.61753939],
       [0.31241225, 0.47884695, 1.        , ..., 0.90746282, 0.69087135,
        0.68596706],
       [0.27859161, 0.45044142, 1.        , ..., 0.94370973, 0.72502998,
        0.72854113],
       ...,
       [       nan,        nan,        nan, ..., 1.        , 1.        ,
        0.95422972],
       [       nan,        nan,        nan, ...,        nan, 1.        ,
        0.97345927],
       [       nan,        nan,        nan, ...,        nan, 1.        ,
        0.22015585]])
Coordinates:
  * st_ocean  (st_ocean) float64 -0.5413 -1.681 -2.94 ... -1.858e+03 -2.007e+03
  * xt_ocean  (xt_ocean) float64 145.1 145.2 145.3 145.4 ... 174.8 174.9 175.0
<xarray.DataArray 'temp_yflux_adv_ym42_01_std_err_mod' (st_ocean: 55, xt_ocean: 300)>
array([[2.64383614e+08, 1.45353039e+08, 0.00000000e+00, ...,
        7.49515937